In [2]:
import json

import numpy as np
import pandas as pd
import pymongo
import scipy
from bson import ObjectId, json_util
from pandas.io.json import json_normalize
from pymongo import MongoClient as Connection
from scipy import sparse, spatial
from scipy.spatial.distance import cdist, pdist, squareform
from sklearn.metrics.pairwise import cosine_similarity

In [3]:

connection = Connection()
db = connection.recipe_db
input_data = db.recipes_test_copy
#data = pd.DataFrame(list(input_data.find()))

data = json.loads(json_util.dumps(input_data.find()))
norm_data = json_normalize(data)

ingredients = json_normalize(data, record_path='ingredients', meta='id', record_prefix='ingredients_', errors='ignore' )
ingredients = pd.DataFrame(ingredients)

#schmeiss alle zutaten raus, die weniger als 4 mal verwendet werden
# ingredients_unique = ingredients[ingredients.groupby('ingredients_id').ingredients_id.transform(len) > 4]

# drop duplicate by a column name
 
# ingredients_unique.drop_duplicates(['ingredients_id'], keep='last')

# ingredients_unique = ingredients_unique.drop(ingredients_unique.columns[[0, 3, 4]], axis=1)

# ingredients_unique['ingredients_id'].value_counts()



In [4]:
norm_data

,categories,name,id,authorId,author,description,prep_time,cook_time,ready_in_time,servings,rating,rating_count,review_count,made_it_count,api_url,url,ingredients,nutritions,_id.$oid
0,"[{'name': 'Vegetarian'}, {'name': 'Casserole'}...",Spinach Enchiladas,59661,662842,SADONIA2,"If you like spinach and Mexican food, you'll l...",20,20,40,4,4.335593,1180,817,2267,https://apps.allrecipes.com/v1/recipes/59661,https://www.allrecipes.com/recipe/59661/spinac...,"[{'id': 16157, 'name': '10 g butter', 'grams':...","[{'name': 'Fat', 'amount': 35.95846, 'unit': '...",5b2257a5ff20008b5a00085d
1,"[{'name': 'Gluten Free'}, {'name': 'Dairy Free...",Stuffed Peppers,16330,25072,BDEGER,Green peppers stuffed with ground beef and ric...,20,60,80,4,4.289096,1605,1111,2687,https://apps.allrecipes.com/v1/recipes/16330,https://www.allrecipes.com/recipe/16330/stuffe...,"[{'id': 3103, 'name': '305 g ground beef', 'gr...","[{'name': 'Fat', 'amount': 9.378057, 'unit': '...",5b2257acff20008b5a00085e
2,"[{'name': 'Chicken'}, {'name': 'Main'}, {'name...",Crispy and Tender Baked Chicken Thighs,235151,2032612,Nicole Burdett,A simple spice blend gives tons of flavor to t...,10,60,70,4,4.640516,1007,786,3923,https://apps.allrecipes.com/v1/recipes/235151,https://www.allrecipes.com/recipe/235151/crisp...,"[{'id': 10536, 'name': 'cooking spray', 'grams...","[{'name': 'Fat', 'amount': 11.88371, 'unit': '...",5b2257b0ff20008b5a00085f
3,"[{'name': 'Quick Easy'}, {'name': 'Chicken'}, ...",Simple Baked Chicken Breasts,240208,196935,Always Cooking Up Something,Simple basic recipe for cooking up a bunch of ...,15,25,40,4,4.606383,282,204,2116,https://apps.allrecipes.com/v1/recipes/240208,https://www.allrecipes.com/recipe/240208/simpl...,"[{'id': 6494, 'name': '4 skinless, boneless ch...","[{'name': 'Fat', 'amount': 9.5803, 'unit': 'g'...",5b2257b4ff20008b5a000860
4,"[{'name': 'Easter'}, {'name': 'Salad'}, {'name...",Delicious Egg Salad for Sandwiches,147103,2309128,wifeyluvs2cook,This is a wonderful-tasting egg salad sandwich...,10,15,35,4,4.618983,1475,1027,3046,https://apps.allrecipes.com/v1/recipes/147103,https://www.allrecipes.com/recipe/147103/delic...,"[{'id': 16317, 'name': '8 eggs', 'grams': 400....","[{'name': 'Fat', 'amount': 31.85464, 'unit': '...",5b2257bdff20008b5a000861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6299,"[{'name': 'BBQ'}, {'name': 'Beef'}, {'name': '...",Miso-Glazed Skirt Steak,228049,8601924,Chef John,This succulent grilled skirt steak recipe woul...,10,8,48,4,4.138889,36,28,82,https://apps.allrecipes.com/v1/recipes/228049,https://www.allrecipes.com/recipe/228049/miso-...,"[{'id': 18888, 'name': '45 ml red wine vinegar...","[{'name': 'Fat', 'amount': 7.365764, 'unit': '...",5b23861bff20008b5a0020f9
6300,"[{'name': 'Slow'}, {'name': 'Lowcarb'}, {'name...",Slow Cooker Herbed Turkey Breast,229659,4290639,Andersonfam09,"The most amazing, flavorful, moist, turkey bre...",15,360,390,4,4.527778,36,29,70,https://apps.allrecipes.com/v1/recipes/229659,https://www.allrecipes.com/recipe/229659/slow-...,"[{'id': 6647, 'name': '3/8 (5 pound) boneless ...","[{'name': 'Fat', 'amount': 13.42641, 'unit': '...",5b23861dff20008b5a0020fa
6301,"[{'name': 'Asian'}, {'name': 'Chicken'}, {'nam...",Chef John's Chicken Satay Burger,236032,8601924,Chef John,"I love chicken satay, that spicy meat on a sti...",15,20,35,4,4.805555,36,26,53,https://apps.allrecipes.com/v1/recipes/236032,https://www.allrecipes.com/recipe/236032/chef-...,"[{'id': 2858, 'name': '35 g peanut butter', 'g...","[{'name': 'Fat', 'amount': 8.863402, 'unit': '...",5b238622ff20008b5a0020fb
6302,"[{'name': 'Chicken'}, {'name': 'Main'}, {'name...",Pineapple Jerk Chicken,245877,17095853,Cans Get You Cooking®,Canned pineapple is harvested and packed at it...,10,20,30,4,4.500000,36,27,152,https://apps.allrecipes.com/v1/recipes/245877,https://www.allrecipes.com/recipe/245877/pinea...,"[{'id': 1650, 'name': '125 g uncooked long-gra...","[{'name': 'Fat', 'amount': 4.347167, 

In [5]:
nutritions = json_normalize(data, record_path='nutritions',
                            meta=['id', 'prep_time', 'rating', 'rating_count', 'ready_in_time', 'review_count'])


In [6]:
nutritions.id.nunique()

6304

In [7]:
v = nutritions.pivot_table(
        index=['id'], 
         columns='name', 
         values='amount'
).reset_index()

v.index.name = v.columns.name = None

v.set_index('id', inplace=True)
v = v.apply(pd.to_numeric)
v = v.fillna(0)
v

,Calcium,Calories,Calories from Fat,Carbohydrates,Cholesterol,Dietary Fiber,Fat,Folate,Iron,Magnesium,Niacin Equivalents,Potassium,Protein,Saturated Fat,Sodium,Sugars,Thiamin,Vitamin A - IU,Vitamin B6,Vitamin C
id,,,,,,,,,,,,,,,,,,,,
6806,90.66800,285.2920,99.71441,34.167180,28.978860,1.442971,11.079380,95.69036,2.295817,16.86080,6.634804,104.79320,11.282430,5.070814,623.4615,2.861161,0.391791,258.77250,0.073362,1.830202
6900,117.79860,105.2702,21.33330,17.988290,1.626667,0.565567,2.370367,39.16360,1.143459,7.30480,1.821021,54.07343,2.842823,0.594948,385.7598,1.721409,0.171691,116.75390,0.017091,0.045053
7198,244.16520,731.5092,325.65230,73.295740,88.693890,4.641466,36.183590,249.14210,5.765659,52.89296,13.513250,509.34610,27.581950,11.506910,1660.2030,2.834244,0.891499,699.95980,0.363192,67.325330
8493,415.52450,454.2830,177.75310,23.831200,203.868500,1.995000,19.750340,63.05025,3.112575,54.35775,23.437800,509.41500,44.076470,9.960631,1107.9110,3.453043,0.174060,578.21420,0.645630,1.912500
8494,58.08641,834.4448,514.68470,4.769545,283.750000,0.636005,57.187190,33.44816,3.723355,84.44925,39.184630,807.96980,71.107120,16.354300,567.9714,2.087409,0.029208,619.52510,1.372439,20.175460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261608,115.27280,324.2800,83.41859,53.913190,0.000000,13.909810,9.268733,233.83750,3.836451,90.64800,4.473349,1234.76200,12.641160,1.209190,1075.4720,5.410336,0.171842,1102.67700,1.279889,137.706800
263611,415.72130,612.4507,291.35620,52.633780,74.500770,3.438973,32.372910,112.82310,3.886419,32.95200,7.449093,214.92120,25.419990,14.229690,1486.6130,4.824386,0.614724,957.93720,0.175574,9.862987
263744,53.87948,297.1487,172.23010,8.851259,68.460000,1.817594,19.136680,42.33592,2.790447,40.50691,8.949356,465.41230,22.367880,6.616539,981.8285,4.866791,0.099264,37.37281,0.413054,24.501440


In [7]:
# data cleansing

# schmeiss alle zutaten raus, die weniger als 5 mal verwendet werden
ingredients_filt = ingredients.groupby('ingredients_id').filter(lambda x: len(x) >= 5)

# drop alls rows with ingredients_id == 0
ingredients_filt = ingredients_filt[ingredients_filt.ingredients_id != 0]

# drop all duplicate
ingredients_filt = ingredients_filt.drop_duplicates(keep='first')


In [8]:
# data cleansing

# schmeiss alle zutaten raus, die weniger als 5 mal verwendet werden
ingredients_filt = ingredients.groupby('ingredients_id').filter(lambda x: len(x) >= 5)

# drop alls rows with ingredients_id == 0
ingredients_filt = ingredients_filt[ingredients_filt.ingredients_id != 0]

# drop all duplicate
ingredients_filt = ingredients_filt.drop_duplicates(keep='first')
ingredients_filt.id.nunique()

6305

In [9]:
id_list = [215014, 8669, 16700, 16354, 12720, 8652, 8887, 51283, 45954, 213742, 14595, 219164, 16348, 143082, 8665, 11758, 223042, 236609, 8778, 65896, 24264, 11679, 141678, 9023]


recipe_db = pd.get_dummies(ingredients_filt['ingredients_id']).groupby(ingredients_filt['id']).apply(max)



In [10]:
recipe_db.index.nunique()

6305

In [11]:
#filtere Rezepte aus den Metadaten anhand ihrerr Zutatenmenge raus. 
not_wanted_recipes = recipe_db[recipe_db.mask(recipe_db==0).count(axis=1) <= 4]
mask = nutritions['id'].isin(not_wanted_recipes.index.values)
not_wanted_recipes

,111,126,257,388,443,445,578,615,629,631,...,24850,24865,25518,25522,26269,26706,26934,27343,27388,27805
id,,,,,,,,,,,,,,,,,,,,,
8496,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8502,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8517,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8559,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8595,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242342,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
242357,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
244458,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:

nutritions_filt = nutritions.loc[~mask]
# nutritions_filt.id.nunique()
v = nutritions_filt.pivot_table(
        index=['id'], 
         columns=['name'],
         values=['amount'],
).reset_index()

v.set_index('id', inplace=True)
nutritions_filt.set_index('id', inplace=True)
nutritions_filt

,name,amount,unit,display_value,percent_daily_value,prep_time,rating,rating_count,ready_in_time,review_count
id,,,,,,,,,,
59661,Fat,35.958460,g,36,55,20,4.33559,1180,40,817
59661,Calories,509.802900,kcal,510,25,20,4.33559,1180,40,817
59661,Cholesterol,95.274040,mg,95,32,20,4.33559,1180,40,817
59661,Sodium,353.601100,mg,354,14,20,4.33559,1180,40,817
59661,Carbohydrates,32.266050,g,32.3,10,20,4.33559,1180,40,817
...,...,...,...,...,...,...,...,...,...,...
244188,Sugars,0.321755,g,0.3,0,10,4.80556,36,1095,30
244188,Potassium,392.877600,mg,393,11,10,4.80556,36,1095,30
244188,Saturated Fat,4.652896,g,4.7,23,10,4.80556,36,1095,30


In [ ]:
#ready_in_time = nutritions_filt.pivot_table(
#        index=['id'], 
#          columns=['ready_in_time'],
#         values=['ready_in_time'],
#).reset_index()

# v.index.name = v.columns.name = None

#ready_in_time.set_index('id', inplace=True)
# ready_in_time

#prep_time = nutritions_filt.pivot_table(
        index=['id'], 
#          columns=['ready_in_time'],
         values=['prep_time'],
#).reset_index()

# v.index.name = v.columns.name = None

#prep_time.set_index('id', inplace=True)
# prep_time

#prep_time[prep_time == 0].count()
#ready_in_time[ready_in_time == 0].count()

In [ ]:

# v.reset_index()
# nutritions_filt.reset_index()
v_new = pd.concat([v, nutritions_filt['prep_time']], axis=1)

# v_new = v_new.set_index('id', inplace=True)
v_new

In [ ]:
#filter alle rezepte, die mehr gleich/als 5 rezepte haben
recipe_db = recipe_db[recipe_db.mask(recipe_db==0).count(axis=1) >= 5]

recipe_db.index.nunique()
user_recipes = recipe_db[recipe_db.index.isin(id_list)]
user_recipes

In [ ]:
### Delta Methode - Rezept zu Rezeptvergleich

# result_1 = pd.DataFrame()

# for index, recipe in recipe_db.iterrows():
#     user_recipes_subtracted = user_recipes.sub(recipe)
#     recipe = pd.Series({'delta':user_recipes_subtracted.abs().values.sum()}, name=index)
#     result_1 = result_1.append(recipe)

# result_1 = result_1.sort_values(by='delta')
#result_1


In [16]:
##Euklidische Distanz - rezept zu rezept vergelich

result_array = cdist(user_recipes, recipe_db,'euclidean')
result_2 = pd.DataFrame(result_array, columns=recipe_db.index.values, index=user_recipes.index.values)
# result_2

result_3 = pd.DataFrame(result_2.sum(), columns=['euclidiean_sum'])
result_3 = result_3.sort_values(by='euclidiean_sum')

result_3

,euclidiean_sum
8652,74.255594
8617,75.439156
56352,75.605380
17146,75.741587
49347,76.223829
...,...
105044,120.302740
239867,120.508061
232359,120.831393
218905,121.438364


In [17]:
##Minkowski Distanz - rezept zu rezept vergelich

result_array = cdist(user_recipes, recipe_db,'minkowski', p = 2)
result_2 = pd.DataFrame(result_array, columns=recipe_db.index.values, index=user_recipes.index.values)
# result_2

result_8 = pd.DataFrame(result_2.sum(), columns=['euclidiean_sum'])
result_8 = result_8.sort_values(by='euclidiean_sum')

result_8

,euclidiean_sum
8652,74.255594
8617,75.439156
56352,75.605380
17146,75.741587
49347,76.223829
...,...
105044,120.302740
239867,120.508061
232359,120.831393
218905,121.438364


In [18]:
##Euklidische Distanz - rezept zu rezept vergelich

result_array = cdist(user_recipes, recipe_db,'jaccard')
result_2 = pd.DataFrame(result_array, columns=recipe_db.index.values, index=user_recipes.index.values)
# result_2

result_6 = pd.DataFrame(result_2.sum(), columns=['jaccard_distance_sum'])
result_6 = result_6.sort_values(by='jaccard_distance_sum')

#result_6

In [19]:
##Euklidische Distanz - rezept zu rezept vergelich

result_array = cdist(user_recipes, recipe_db,'cityblock')
result_2 = pd.DataFrame(result_array, columns=recipe_db.index.values, index=user_recipes.index.values)
# result_2

result_7 = pd.DataFrame(result_2.sum(), columns=['manhattan_distance (delta-Methode)'])
result_7 = result_7.sort_values(by='manhattan_distance (delta-Methode)')

#result_7

In [ ]:
##Euklidische Distanz - Userpräferenz
#1. Array mit user Rezepten zusammenstellen (am besten mit index, sodass jedes Rezept eine feste nummer hat)
#2. Aus allen Rezepten des Arrays einen Userpräferenz Vektor bilden (Rezeptpräsenz wird aufadiert)

# user_pref = pd.DataFrame(index=['user_pref'])
# user_pref.loc['user_pref'] = user_recipes.sum()
# distance_array = cdist(user_pref.loc[['user_pref']], recipe_db, metric='euclidean')
# result_4 = pd.DataFrame(distance_array, columns=recipe_db.index.values, index=['euclidiean_user_pref'])
# result_4 = result_4.T.sort_values(by='euclidiean_user_pref')
# result_4

In [20]:

result_array = cosine_similarity(user_recipes, recipe_db)
result_2 = pd.DataFrame(result_array, columns=recipe_db.index.values, index=user_recipes.index.values)
result_5 = pd.DataFrame(result_2.sum(), columns=['cosine_distance_sum'])
result_5 = result_5.sort_values(by='cosine_distance_sum')
result_5 = result_5.sort_values(ascending=False, by='cosine_distance_sum')

In [ ]:
display_side_by_side(result_7, result_3, result_5, result_6)


In [ ]:

# result_array = cosine_similarity(user_recipes, recipe_db)
# result_2 = pd.DataFrame(result_array, columns=recipe_db.index.values, index=user_recipes.index.values)
# result_3 = pd.DataFrame(result_2.sum(), columns=['distance_sum'])
# result_3 = result_3.sort_values(by='distance_sum')
#result_3.sort_values(ascending=False, by='distance_sum')

In [22]:
top10 = result_6[:10].index.values

#überschneidung jaccard und euklidische distanz top 100 
jac = pd.Series(result_6[:100].index.values)
euc = pd.Series(result_3[:100].index.values)

pd.Series(list(set(jac).intersection(set(euc))))

0     8611
1    14534
2     8652
3     8560
4    55860
5    11732
6    78142
dtype: int64